In [1]:
from datetime import datetime

import qnexus as qnx

from pytket import Circuit



In [2]:
my_project = qnx.project.get_only(name_like="VanyaTest")

qnx.context.set_active_project(my_project)

my_job_name_prefix = datetime.now()

## Compile Jobs

In [4]:
c = Circuit(2,2).H(0).CX(0,1).measure_all()

my_circuit_ref = qnx.circuit.upload(
    circuit=c,
    name="A circuit 17 May",
)

In [5]:
my_circuit_ref.df()

,name,description,created,modified,project,id
0,A circuit 17 May,None,2024-05-20 13:29:03.859905+00:00,2024-05-20 13:29:03.859905+00:00,VanyaTest,9a0216ec-fc88-4f09-8282-d2cbeb7b2bb5


In [6]:
compile_job_ref = qnx.compile(
    circuits=[my_circuit_ref],
    name=f"{my_job_name_prefix}_compile",
    optimisation_level=1,
    target=qnx.QuantinuumConfig(device_name="H1-1SC")
)

In [11]:
compile_results = qnx.job.results(compile_job_ref)

In [6]:
qnx.job.wait_for(compile_job_ref)

JobStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='The job is completed.', error_detail=None, completed_time=datetime.datetime(2024, 5, 20, 13, 16, 45, 809847, tzinfo=datetime.timezone.utc), queued_time=None, submitted_time=datetime.datetime(2024, 5, 20, 13, 16, 36, 381994, tzinfo=datetime.timezone.utc), running_time=datetime.datetime(2024, 5, 20, 13, 16, 36, 443329, tzinfo=datetime.timezone.utc), cancelled_time=None, error_time=None, queue_position=None)

In [7]:
# Pretending we lost the compile_job_ref variable, we can retrieve it using filters

job_matches = qnx.job.get(name_like=f"{my_job_name_prefix}_compile")

my_job = job_matches.list()[0]

my_job.df()

,name,description,created,modified,job_type,last_status,project,id
0,2024-05-20 14:16:33.258027_compile,None,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00+00:00,JobType.COMPILE,StatusEnum.COMPLETED,unimplemented,0c443704-425e-454d-a949-7689719b4a07


In [11]:
# get results
compile_results = qnx.job.results(compile_job_ref)

compile_results.df()

,name,description,created,modified,project,id
0,A circuit 17 May-compilation,,2024-05-20 13:16:45.521845+00:00,2024-05-20 13:16:45.521845+00:00,VanyaTest,e20e63cb-e463-48b6-b14c-519e1ee47cef


In [12]:
compile_results[0].df()

,name,description,created,modified,project,id
0,A circuit 17 May-compilation,,2024-05-20 13:16:45.521845+00:00,2024-05-20 13:16:45.521845+00:00,VanyaTest,e20e63cb-e463-48b6-b14c-519e1ee47cef


In [13]:
compile_results[0].get_passes().df()

,pass name,input,output,id
0,DecomposeBoxes,A circuit 17 May,A circuit 17 May,a1ec7fe2-6913-434d-a79b-d40a2a6c6800
1,CustomPass,A circuit 17 May,A circuit 17 May,1c945435-684f-4640-b0c2-e5ccfdac88e5
2,SynthesiseTK,A circuit 17 May,A circuit 17 May-QuantinuumBackend-2,45e7cbb3-2ffb-41c8-8ed4-115948817ad8
3,NormaliseTK2,A circuit 17 May-QuantinuumBackend-2,A circuit 17 May-QuantinuumBackend-2,f8b67d1f-3a5e-406f-8dec-8da5df38e0eb
4,DecomposeTK2,A circuit 17 May-QuantinuumBackend-2,A circuit 17 May-QuantinuumBackend-3,11f1df6a-ced5-4d98-8014-9bbf5415824e
5,RebaseCustomViaTK2,A circuit 17 May-QuantinuumBackend-3,A circuit 17 May-QuantinuumBackend-4,c4aaf031-ce0e-46fa-9d44-f36120801e67
6,ZZPhaseToRz,A circuit 17 May-QuantinuumBackend-4,A circuit 17 May-QuantinuumBackend-4,6f5dfc29-065b-4b7d-a65a-261b3b7e86cc
7,RemoveRedundancies,A circuit 17 May-QuantinuumBackend-4,A circuit 17 May-QuantinuumBackend-4,8afceb5a-a450-428b-9f27-93a121fd780d
8,SquashRzPhasedX,A circuit 17 May-QuantinuumBackend-4,A circuit 17 May-QuantinuumBackend-5,b5ad36ff-8639-4a11-a564-3bb315cf531d
9,RemoveRedundancies,A circuit 17 May-QuantinuumBackend-5,A circuit 17 May-QuantinuumBackend-final,c0d430ad-2bdd-402a-84de-e08728f3e9bc


In [14]:
compile_results[0].get_output().download_circuit()

[PhasedX(3.5, 0.5) q[0]; PhasedX(0.5, 0.5) q[1]; ZZPhase(0.5) q[0], q[1]; Measure q[0] --> c[0]; PhasedX(0.5, 0) q[1]; Measure q[1] --> c[1]; ]

In [15]:
passes = compile_results[0].get_passes()


passes.df()

,pass name,input,output,id
0,DecomposeBoxes,A circuit 17 May,A circuit 17 May,a1ec7fe2-6913-434d-a79b-d40a2a6c6800
1,CustomPass,A circuit 17 May,A circuit 17 May,1c945435-684f-4640-b0c2-e5ccfdac88e5
2,SynthesiseTK,A circuit 17 May,A circuit 17 May-QuantinuumBackend-2,45e7cbb3-2ffb-41c8-8ed4-115948817ad8
3,NormaliseTK2,A circuit 17 May-QuantinuumBackend-2,A circuit 17 May-QuantinuumBackend-2,f8b67d1f-3a5e-406f-8dec-8da5df38e0eb
4,DecomposeTK2,A circuit 17 May-QuantinuumBackend-2,A circuit 17 May-QuantinuumBackend-3,11f1df6a-ced5-4d98-8014-9bbf5415824e
5,RebaseCustomViaTK2,A circuit 17 May-QuantinuumBackend-3,A circuit 17 May-QuantinuumBackend-4,c4aaf031-ce0e-46fa-9d44-f36120801e67
6,ZZPhaseToRz,A circuit 17 May-QuantinuumBackend-4,A circuit 17 May-QuantinuumBackend-4,6f5dfc29-065b-4b7d-a65a-261b3b7e86cc
7,RemoveRedundancies,A circuit 17 May-QuantinuumBackend-4,A circuit 17 May-QuantinuumBackend-4,8afceb5a-a450-428b-9f27-93a121fd780d
8,SquashRzPhasedX,A circuit 17 May-QuantinuumBackend-4,A circuit 17 May-QuantinuumBackend-5,b5ad36ff-8639-4a11-a564-3bb315cf531d
9,RemoveRedundancies,A circuit 17 May-QuantinuumBackend-5,A circuit 17 May-QuantinuumBackend-final,c0d430ad-2bdd-402a-84de-e08728f3e9bc


## Execute Jobs

In [16]:
compiled_circuits = [item.get_output() for item in compile_results]

In [17]:
compiled_circuits

[CircuitRef(id=UUID('4a77c615-5972-4b05-8294-20d4fcd7605e'), annotations=Annotations(name='A circuit 17 May-QuantinuumBackend-final', description=None, properties=OrderedDict(), created=datetime.datetime(2024, 5, 20, 13, 16, 45, 471243, tzinfo=TzInfo(UTC)), modified=datetime.datetime(2024, 5, 20, 13, 16, 45, 471243, tzinfo=TzInfo(UTC))), project=ProjectRef(id=UUID('6ed750a5-de0c-4879-a771-2c1f3f746d18'), annotations=Annotations(name='VanyaTest', description='', properties=OrderedDict(), created=datetime.datetime(2024, 1, 29, 13, 38, 27, 351099, tzinfo=TzInfo(UTC)), modified=datetime.datetime(2024, 1, 29, 13, 38, 27, 351099, tzinfo=TzInfo(UTC))), contents_modified=datetime.datetime(2024, 5, 20, 13, 16, 45, 809847, tzinfo=TzInfo(UTC))))]

In [18]:
execute_job_ref = qnx.execute(
    circuits=compiled_circuits,
    name=f"{my_job_name_prefix}1_execute",
    n_shots=[100]* len(compiled_circuits),
    target=qnx.QuantinuumConfig(device_name="H1-1SC"),
)

In [19]:
qnx.job.wait_for(execute_job_ref)

JobStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='The job is completed.', error_detail=None, completed_time=datetime.datetime(2024, 5, 20, 13, 18, 11, 117318, tzinfo=datetime.timezone.utc), queued_time=None, submitted_time=datetime.datetime(2024, 5, 20, 13, 17, 30, 533858, tzinfo=datetime.timezone.utc), running_time=None, cancelled_time=None, error_time=None, queue_position=None)

In [20]:
execute_results = qnx.job.results(execute_job_ref)

execute_results.df()

,name,description,created,modified,project,id
0,2024-05-20 14:16:33.2580271_execute,None,None,None,VanyaTest,dd11ab28-ef16-4a7c-b1a8-cbc8e3ffaa40


In [21]:
execute_results[0].download_backend_info()

BackendInfo(name='QuantinuumBackend snapshot', device_name='H1-1SC', version='0.33.0', architecture=<tket::FullyConnected, nodes=20>, gate_set={<OpType.Measure: 66>, <OpType.Reset: 68>, <OpType.Rz: 36>, <OpType.PhasedX: 71>, <OpType.Barrier: 8>, <OpType.ZZMax: 73>, <OpType.ClassicalExpBox: 107>, <OpType.ZZPhase: 76>, <OpType.TK2: 44>, <OpType.WASM: 14>, <OpType.SetBits: 15>, <OpType.CopyBits: 16>, <OpType.RangePredicate: 17>, <OpType.ExplicitPredicate: 18>, <OpType.ExplicitModifier: 19>, <OpType.MultiBit: 20>}, n_cl_reg=0, supports_fast_feedforward=True, supports_reset=True, supports_midcircuit_measurement=True, all_node_gate_errors=None, all_edge_gate_errors=None, all_readout_errors=None, averaged_node_gate_errors=None, averaged_edge_gate_errors=None, averaged_readout_errors=None, misc={'cl_reg_width': 32, 'max_n_shots': 10000, 'options': {}, 'system_type': 'syntax checker', 'wasm': True})

In [22]:
execute_results[0].download_result()

BackendResult(q_bits={},c_bits={c[0]: 0, c[1]: 1},counts=None,shots=[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]],state=None,unitary=None,density_matrix=None)

In [ ]:
# Example of a job erroring

In [ ]:
execute_job_ref = qnx.execute(
    circuits=[my_circuit_ref],
    name=f"error_execute_{datetime.now()}",
    n_shots=[100],
    target=qnx.QuantinuumConfig(device_name="H1-1LE")
)

In [ ]:
qnx.job.wait_for(execute_job_ref)